# A Practical Guide to Computing Diffusivity of Amorphous Materials Using VASP

This guide provides a detailed procedure with all necessary scripts to compute diffusivity of any amorphous material using the VASP package.
For scientific background, see [Angew. Chem. Int. Ed. 2024, 63, e202315628](https://onlinelibrary.wiley.com/doi/full/10.1002/anie.202315628), and [ACS Appl. Mater. Interfaces 2020, 12, 35748](https://pubs.acs.org/doi/10.1021/acsami.0c10000). We will use $LiTaCl_6$ in [Angew. Chem. Int. Ed. 2024, 63, e202315628](https://onlinelibrary.wiley.com/doi/full/10.1002/anie.202315628) as the material, combined with methods from [ACS Appl. Mater. Interfaces 2020, 12, 35748](https://pubs.acs.org/doi/10.1021/acsami.0c10000) to achieve efficient calculation of diffusivity.

First of all, you need to install some packages. 

Download and compile [packmol](https://m3g.github.io/packmol/). 

The code was tested in a conda environment and that's the recommended way to use the code. Suppose you have conda installed/loaded, do the following to create an environment for this project:

<code>conda create --name vaspenv python=3.11 numpy scipy ase pymatgen</code>. 

Then type <code>conda activate vaspenv</code> to use the environment.

The first step is to melt and equilibrate the material above its melting point but below its boiling point. We follow [ACS Appl. Mater. Interfaces 2020, 12, 35748](https://pubs.acs.org/doi/10.1021/acsami.0c10000) to create a randomly packed initial structure using the packmol package.

<code>cd initialStructure</code> 

Modify the user input section of <code>getInitialStructure.py</code> for your own system.

```python
# ----- user data -------------------------------------------------------------
molar_masses = {"Li": 6.9410, "Ta": 180.94788, "Cl": 35.45}     # g mol⁻¹
stoich       = {"Li": 1, "Ta": 1, "Cl": 6}                      # LiTaCl₆
density      = 2.96                      # g cm⁻³
box_diam     = 12.0                      # Å
packmolPath = "/pscratch/sd/x/xlliu9/Software/packmol/packmol"
# ---------------------------------------------------------------------------
```
The user needs to supply molar masses and stoichiometry for each element in the system, as well as density, diameter of the cubic box, and the location of packmol package. Note the code will create a cubic box with diameters slightly greater than <code>box_diam</code>.
Run the code and you will get a <code>POSCAR</code> file in the same directory. This is the starting geometry for our simulations.

We then equilibrate the structure at a suitable temperature. For $LiTaCl_6$, we choose 1500 K. Copy the POSCAR file to <code>eql_1500</code> and prepare other files. To better reach equilibrium, a Langevin thermostat should be used. Run <code>pltAll.py</code> frequently to monitor the status of MD. 

Once it reached equilibrium, we need to determine its density at 1500 K. We do so by iteratively scale the lattice parameters according to:

$\frac{\Delta V}{V} \approx -\frac{\Delta P}{K}$, 

where $V$ is the volume of the cell, $P$ is pressure, and $K$ is bulk modulus. We approximate $K$ by $100 GPa$.

